#**LLama 2**

#**Установим и импортируем нужные библиотеки**


In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 10.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 10.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 57.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 40.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 7.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-6hsowuob/overlay/local/bin
    changing mode of /tmp/pip-build-env-6hsowuob/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-6hsowuob/overlay/

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [3]:
from huggingface_hub import hf_hub_download


In [4]:
from llama_cpp import Llama


#**Загрузим модель**


In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [12]:
import pandas as pd

#**Получим данные**

In [ ]:
from google.colab import files

# Загрузить файл с локального компьютера
uploaded = files.upload()

In [16]:

data=pd.read_csv("data.csv")
data.head()
data.columns
data.message

0      Привет! Я хочу заказать букет роз доставить в ...
1      Привет, хочу заказать доставку цветов в Новоси...
2      Здравствуйте, сколько стоит доставка цветов в ...
3      Здравствуйте, сколько стоит доставка цветов в ...
4      Привет! Хочу заказать цветы на день рождения п...
                             ...                        
115    Привет, доставите букет роз в г. екат? Срочно ...
116    Привет, сколько стоит доставка цветов в г. ниж...
117    Хочу цветы в Новосибирске. Скорее быстро сдела...
118    Здравствуйте! Хочу заказать букет роз с достав...
119    Здравствуйте. Я хочу заказать цветы в Москву н...
Name: message, Length: 120, dtype: object

#**Получим ответы**

In [25]:

def get_answer(data):
  gens=[]
  for i in data.message:
    prompt = f"{i}"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

    USER: Перед тобой сообщение человека который хочет заказать цветы.О каком городе в сообщении идет речь? Напиши только официальное название города с большой буквы.Только название и ничего больше
    {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
    gens.append(response["choices"][0]["text"])
  return gens



In [26]:
get_answer(data[:1])

Llama.generate: prefix-match hit


['SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\n    USER: Перед тобой сообщение человека который хочет заказать цветы.О каком городе в сообщении идет речь? Напиши только официальное название города с большой буквы.Только название и ничего больше\n    Привет! Я хочу заказать букет роз доставить в Питер сегодня до 15:00. Можно у вас такое оформить? \n    \n    ASSISTANT:\n    \n    Здравствуйте! Хорошо, что вы обратились к нам с заказом цветов. Наш сервис доставки цветов работает в Санкт-Петербурге. Мы можем оформить ваш заказ и доставить букет роз сегодня до 15:00. Для этого пожалуйста предоставьте нам более подробную информацию, такую как адрес доставки и телефонный номер для связи.']

Впринципе можно было бы обрезать после assistant но 13b llama все равно не понимает что мы от нее хотим